In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO  # Formuoja str tipo duomenis į StringIO objektą vėlesniam naudojimui
from tabulate import tabulate
from urllib.parse import urljoin  # Gražiai sujungia bazinę nuorodą su 'endpoint'
import re
import datetime as dt

***Funkcijos***

In [2]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

def lenteliu_formavimas(lenteles, data, spausdinti=False): # Funkcija naudojama "išvirus sriubą" su Beautiful soup
    if lenteles:
        duomenys = [data]
        for i in range(len(lenteles)):
            lenteles_html = str(lenteles[i]) # Sriubos elementą paverčiam į str tipą
            df = pd.read_html(StringIO(lenteles_html), thousands=None, header=0)[0] # Str turinį su 'StringIO' imituoja kaip failą, kad pandas suformmatuotų jį į DataFrame
            df.dropna(how='all', axis=1, inplace=True)
            df.dropna(how='all', axis=0, inplace=True)
            df.fillna('', inplace=True)

            # Keičiame , į . kad vėliau būtų paprasta skaičiuoti arba atvaizduoti kaip float tipą jei prireiks
            for column in df.columns:
                df[column] = df[column].astype(str)
                df[column] = df[column].str.replace(',', '.', regex=False)
            
            if spausdinti:
                print(f"{data} Lentelė {i+1}")
                print(tabulate(df, headers='keys', tablefmt='grid'))
                print("\n")
            duomenys.append(df)
        return duomenys
    else:
        print("Lentelių nerasta.")


def prideti_lentele(sarasas):  # Funkciją naudoti galima įvedant duomenis rankiniu būdu
    tarifas = input("Įveskite tarifą (pvz., Standartinis Planas): ")
    dedamoji = input("Įveskite dedamosios pavadinimą: ")
    kaina = float(input("Įveskite kainą be PVM: "))
    pvm = 1.21
    kaina_su_pvm = kaina * pvm
    sarasas.append({'Tarifas': tarifas, 'Dedamoji': dedamoji, 'Kaina be PVM': kaina, 'Kaina su PVM': kaina_su_pvm})


***Nuorodų surinkimas į list.***

Nuorodas renkuosi dėl skirtingų jų pavadinimų bei skirtingų informacijos padavimo būdų.  
Formuoju dictionary, kad suporuoti datas su lentelėmis

'Pastaba, atnaujinimas įvyko liepos 22d.'

In [3]:
link = 'https://www.regula.lt/elektra/Puslapiai/tarifai/visuomeniniai-tarifai-eso-galiojantys.aspx' # Pradinė nuoroda el. kainų filtravimui
pradine_sriuba = get_soup(link)

pagr_link = 'https://www.regula.lt' # Nuorodų 'lipdymui'
link_sar = []
for link in pradine_sriuba.find_all('a'): # 'a' ieškomos visos eilutės kuriose yra po nuorodą
    href = link.get('href') # href yra Hypertext Reference, žymė kuri pradeda nuorodą ('link')
    if href:
        full_url = urljoin(pagr_link, href) # Gražiai sujungia bazinę nuorodą su 'endpoint'
        text = link.text.strip() # Paliekamas tik tekstas susijęs su nuoroda
        if 'visuomeniniai tarifai' in text.lower(): # Filtruoju nuorodas pagal raktinius žodžius
            dict = {text: full_url}
            link_sar.append(dict)
            print(f'{text}, URL: {full_url}') # Spausdinti rezultatus, pasitikrinti output ar viskas tinka



Visuomeniniai tarifai (UAB „Ignitis“), URL: https://www.regula.lt/elektra/Puslapiai/tarifai/visuomeniniai-tarifai-eso-galiojantys.aspx
​Visuomeniniai tarifai, galiojantys iki 2024-03-31, URL: https://www.regula.lt/elektra/Puslapiai/tarifai/2023/visuomeniniai-tarifai-galioje-iki-2024-03-31.aspx
Visuomeniniai tarifai, galioję iki 2023-12-31​, URL: https://www.regula.lt/elektra/Puslapiai/tarifai/2023/Visuomeniniai-tarifai-galioje-iki-2023-12-31.aspx
Visuomeniniai tarifai, galioję iki 2023-07-01​, URL: https://www.regula.lt/elektra/Puslapiai/tarifai/2023/visuomeniniai-tarifai-galioje-iki-2023-07-01.aspx
Visuomeniniai tarifai, ​galioję iki 20​2​3-03-01​​​, URL: https://www.regula.lt/elektra/Puslapiai/tarifai/2023/visuomeniniai-tarifai-iki-2023-03-01.aspx
Visuomeniniai tarifai, galioję iki 2022-12-31​, URL: https://www.regula.lt/elektra/Puslapiai/tarifai/visuomeniniai-tarifai-galioje-iki-2022-gruodzio-31.aspx
Visuomeniniai tarifai, galioję iki 2021-12-31​, URL: https://www.regula.lt/elektra/

Pasitikrinu informaciją iš nuorodų:  
Ar yra lentelės, jei yra atsipausdinti rezultatus.  
Jei lentelių nerasta atspausdinu nuorodą kurioje lentelės nerastos arba pateiktos neteisingai.  
Rankiniu būdu vėliau suvesti duomenis.


In [4]:
spausdinti_lenteles = True # Jeigu reikia atvaizdavimo output pasižiūrėti kokią informaciją pavyko gauti
duomenu_sar = [] # Suformuojamas duomenų sąrašas
datos_tvarkymui = [] # Iš kurios datos duomenis reikės tvarkyti rankiniu būdu
for el in link_sar: # Imu iš sąrašo indeksą su elementu ('el'), elemente yra key: value įrašyta informacija
    for text, link in el.items():       
        sriuba = get_soup(link)
        lenteles = sriuba.find_all('table') # Daugiskaita, nes lentelių gali būti ne viena
        clean_text = text.replace('\u200b', '')  # Iš html atsiradusi vertė '\n200b' ją reikia pašalinti, kad galėčiau filtruoti su ReGex

        # Regex pagalba filtruojama data YYYY-MM-DD
        pattern = r'\d{4}-\d{2}-\d{2}' # d - digits žymė ir kiek skaitmenų paimti turi
        match = re.search(pattern, clean_text)
        if match:
            date = match.group()
            print('Kainos suformavimo data:', date)
        elif 'Ignitis' in clean_text:
            date = '2024-07-01'
            print('Kainos suformavimo data:', date)
        
        if len(lenteles) == 0:
            print('Tvarkyti rankiniu būdu šią nuorodą:\n', link)
            datos_tvarkymui.append(date)
            continue

        print(f"Kiek lentelių randu: {len(lenteles)}")
        df_lenteles = lenteliu_formavimas(lenteles, date, spausdinti_lenteles)
        # lenteliu_formavimas(lenteles, spausdinti_lenteles)
        duomenu_sar.append(df_lenteles)



Kainos suformavimo data: 2024-07-01
Kiek lentelių randu: 4
2024-07-01 Lentelė 1
+----+--------------------------+--------------------------------+-----------------------------------+
|    | ŽĮ ​​                      | ​                               |   2024 m. II pusm.  ct/kWh su PVM |
+====+==========================+================================+===================================+
|  0 | Viena  laiko zona        | Standartinis                   |                             0.197 |
+----+--------------------------+--------------------------------+-----------------------------------+
|  1 | Dvi laiko zonos ​​         | Dieninė  dedamoji              |                             0.225 |
+----+--------------------------+--------------------------------+-----------------------------------+
|  2 |                          | Naktinė  dedamoji              |                             0.133 |
+----+--------------------------+--------------------------------+-------------------------

Info surinkimas iš nuorodos kurioje lentelių nerasta (jos atvaizduotos jpg, png ar kitu formatu)

In [5]:
pirma_lent = [
    # Standartinis Planas
    {'Tarifas': 'Standartinis Planas', 'Dedamoji': 'Pastovioji dedamoji (EUR/mėn.)', 'Kaina be PVM': 2.48, 'Kaina su PVM': 3.00},
    {'Tarifas': 'Standartinis Planas', 'Dedamoji': 'Energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.138, 'Kaina su PVM': 0.167},
    
    # Namai Planas
    {'Tarifas': 'Namai Planas', 'Dedamoji': 'Pastovioji dedamoji (EUR/mėn.)', 'Kaina be PVM': 4.96, 'Kaina su PVM': 6.00},
    {'Tarifas': 'Namai Planas', 'Dedamoji': 'Energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.125, 'Kaina su PVM': 0.151},
    
    # Namai Plius Planas
    {'Tarifas': 'Namai plius Planas', 'Dedamoji': 'Pastovioji dedamoji (EUR/mėn.)', 'Kaina be PVM': 2.48, 'Kaina su PVM': 3.00},
    {'Tarifas': 'Namai plius Planas', 'Dedamoji': 'Dieninė energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.156, 'Kaina su PVM': 0.189},
    {'Tarifas': 'Namai plius Planas', 'Dedamoji': 'Naktinė energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.102, 'Kaina su PVM': 0.124},
    
    # Išmanusis Planas
    {'Tarifas': 'Išmanusis Planas', 'Dedamoji': 'Pastovioji dedamoji (EUR/mėn.)', 'Kaina be PVM': 4.96, 'Kaina su PVM': 6.00},
    {'Tarifas': 'Išmanusis Planas', 'Dedamoji': 'Dieninė energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.141, 'Kaina su PVM': 0.171},
    {'Tarifas': 'Išmanusis Planas', 'Dedamoji': 'Naktinė energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.095, 'Kaina su PVM': 0.115},
    
    # Keturių laiko zonų Planas
    {'Tarifas': 'Keturių laiko zonų Planas', 'Dedamoji': 'Nakties energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.087, 'Kaina su PVM': 0.105},
    {'Tarifas': 'Keturių laiko zonų Planas', 'Dedamoji': 'Ryto energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.098, 'Kaina su PVM': 0.118},
    {'Tarifas': 'Keturių laiko zonų Planas', 'Dedamoji': 'Dienos energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.126, 'Kaina su PVM': 0.153},
    {'Tarifas': 'Keturių laiko zonų Planas', 'Dedamoji': 'Vakaro energijos kaina (EUR/kWh)', 'Kaina be PVM': 0.150, 'Kaina su PVM': 0.182},
]

antra_lent = [
    # Lentele iš vidutinės įtampos tinklų
    {"Tarifas": "Vienos laiko zonos tarifas", "Dedamoji": "Vienos laiko zonos energijos dedamoji EUR/kWh", "Kaina be PVM": 0.106, "Kaina su PVM": 0.128},
    {"Tarifas": "Dviejų laiko zonų tarifas", "Dedamoji": "Dieninė energijos dedamoji EUR/kWh", "Kaina be PVM": 0.121, "Kaina su PVM": 0.146},
    {"Tarifas": "Dviejų laiko zonų tarifas", "Dedamoji": "Naktinė, Šeštadienio ir sekmadienio energijos dedamoji EUR/kWh", "Kaina be PVM": 0.083, "Kaina su PVM": 0.100}
]

df_1 = pd.DataFrame(pirma_lent)
df_2 = pd.DataFrame(antra_lent)
print(tabulate(df_1, headers='keys', tablefmt='grid', showindex=False))
print(tabulate(df_2, headers='keys', tablefmt='grid', showindex=False))

+---------------------------+-----------------------------------+----------------+----------------+
| Tarifas                   | Dedamoji                          |   Kaina be PVM |   Kaina su PVM |
+===========================+===================================+================+================+
| Standartinis Planas       | Pastovioji dedamoji (EUR/mėn.)    |          2.48  |          3     |
+---------------------------+-----------------------------------+----------------+----------------+
| Standartinis Planas       | Energijos kaina (EUR/kWh)         |          0.138 |          0.167 |
+---------------------------+-----------------------------------+----------------+----------------+
| Namai Planas              | Pastovioji dedamoji (EUR/mėn.)    |          4.96  |          6     |
+---------------------------+-----------------------------------+----------------+----------------+
| Namai Planas              | Energijos kaina (EUR/kWh)         |          0.125 |          0.151 |


In [6]:
for i, el in enumerate(datos_tvarkymui):
    duomenu_sar.append([el, df_1, df_2])

In [7]:
print(duomenu_sar[15])

['2022-12-31',                       Tarifas                           Dedamoji  \
0         Standartinis Planas     Pastovioji dedamoji (EUR/mėn.)   
1         Standartinis Planas          Energijos kaina (EUR/kWh)   
2                Namai Planas     Pastovioji dedamoji (EUR/mėn.)   
3                Namai Planas          Energijos kaina (EUR/kWh)   
4          Namai plius Planas     Pastovioji dedamoji (EUR/mėn.)   
5          Namai plius Planas  Dieninė energijos kaina (EUR/kWh)   
6          Namai plius Planas  Naktinė energijos kaina (EUR/kWh)   
7            Išmanusis Planas     Pastovioji dedamoji (EUR/mėn.)   
8            Išmanusis Planas  Dieninė energijos kaina (EUR/kWh)   
9            Išmanusis Planas  Naktinė energijos kaina (EUR/kWh)   
10  Keturių laiko zonų Planas  Nakties energijos kaina (EUR/kWh)   
11  Keturių laiko zonų Planas     Ryto energijos kaina (EUR/kWh)   
12  Keturių laiko zonų Planas   Dienos energijos kaina (EUR/kWh)   
13  Keturių laiko zonų Planas   V

In [8]:
import pickle
with open('Data_scrap.pkl', 'wb') as file:
    pickle.dump(duomenu_sar, file)


Alternatyvus duomenų suvedimas rankiniu būdu (panaudojant funkciją iš aukščiau):

In [9]:
# pirma_lent = []
# antra_lent = []

# while True:
#     prideti_lentele(pirma_lent)
#     more = input("Ar norite pridėti kitą įrašą? (taip/ne): ").lower()
#     if more != 'taip':
#         break

# df_1 = pd.DataFrame(pirma_lent)
# df_2 = pd.DataFrame(antra_lent)
# print(tabulate(df_1, headers='keys', tablefmt='grid', showindex=False))
# print(tabulate(df_2, headers='keys', tablefmt='grid', showindex=False))